# Lisp Metacircula Interpreter

In [1]:
(require '[clojure.test :refer [is]])

nil

## Main Interpreter Function

### Special Forms in our Lisp Interpreter

- `QUOTE: (QUOTE datum)`
- `IF: (IF condition then-part else-part)`
- `LAMBDA: (LAMBDA (param1 param2 ...) body)`
- `LABEL: (LABEL name expr)`

In [2]:
(declare $eval)

#'user/$eval

In [3]:
(defn $eval-variable
  [expr env]
  (if (contains? env expr)
    (env expr)
    (throw (RuntimeException. 
            (str "Variable " expr " not found in current scope!")))))

#'user/$eval-variable

In [4]:
(defn $eval-list
  [expr env]
  (case (first expr)
    nil ()
    QUOTE (second expr)
    IF (let [[_ condition then-part else-part] expr] 
         (if ($eval condition env)
           ($eval then-part env)
           ($eval else-part env)))))

#'user/$eval-list

In [5]:
(defn $eval
  [expr env]
  (cond
    (symbol? expr) ($eval-variable expr env)
    (list? expr) ($eval-list expr env)
    :else expr))

#'user/$eval

In [6]:
;; Test that ordinary values evaluate to themselves.
(is (= 42 ($eval 42 {})))
(is (= true ($eval true {})))
(is (= "hello" ($eval "hello" {})))
(is (= nil ($eval nil {})))

true

In [7]:
;; Test that symbols represent bindings in the given scope (env).
(is (= 7 ($eval 'x '{x 7})))
(is (= 10 ($eval 'y '{x 7, y 10})))
(try
  ($eval 'z '{x 7, y 10})
  (is false)
  (catch RuntimeException e (is true)))

true

In [8]:
(is (= 'x ($eval '(QUOTE x) '{x 5})))
(is (= '(+ 2 3) ($eval '(QUOTE (+ 2 3)) {})))
(is (= 42 ($eval '(QUOTE 42) {})))

true

In [9]:
(is (= 2 ($eval '(IF 1 2 3) {})))
(is (= 3 ($eval '(IF nil 2 3) {})))
(is (= 10 ($eval '(IF x y z) '{x 5, y 10, z 20})))
(is (= 20 ($eval '(IF x y z) '{x false, y 10, z 20})))

true